## Validate algorithm that Finds Eyes in images.

An algorithm exists that finds many fiducial points (landmark points) in a human MRI brain scan.  The most common failure of this algorithm is an inability to find the Eye lanndmark points in the first stage of the algorithm.  We currently have over 12,000 brain scans processed across 7 different projects.

We need to visually inspect eveery data set to evaluate if the eyes are properly identified with a landmark point approimately in the center of the eye (any landmark that is near the middle of the eye is acceptable.)

The eyes are labeled as `RE` for the right eye and `LE` fro the left eye.  The eyes are approximately `30mm` diameter.  The images do not all have the spaceing for the sampling.

The Landmark files are simple ASCII files that can be easily read with the pandas toolikt

```
# Markups fiducial file version = 4.6
# CoordinateSystem = 0
# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID
vtkMRMLMarkupsFiducialNode_0,0,0,0,0,0,0,1,1,1,0,AC,,
vtkMRMLMarkupsFiducialNode_1,0.00009494799228537687,-30.201098915930984,-44.52037286965537,0,0,0,1,1,1,0,BPons,,
```

1. The HTML report should have 2D RGB snapshot images written in `.png` format of a region around the eye (1 image for left eye, 1 image for right eye) at the location of the eye fiducial.
1. Rescale each image into a range of values from 0-255.
1. Use an identity transform to resample the original image into a physical space with origin at ( $LE_{L} - 15mm$, $LE_{P}-15mm$, $LE_{S}$ ), with isotropic voxels and a $30mm \times 30mm \times 1voxel$ field of view, and identity direction cosing.  (Remember $_{LPS}$ stands for the DICOM standard $L$eft, $P$osterior, $S$uperior designations This image should be an unsiged char greyscale scalar image.  
1. Convert the 3D image into a 2D image (https://itk.org/Doxygen/html/classitk_1_1ExtractImageFilter.html) that is again a scalar image.
1. Convert the 2D scalar image into a 3D RGB imaage where the R,G, &B channels are all the same values
1. Change the pixel at the index location to pure Red for the Right Eye, and pure Blue for the Left Eye

In [46]:
import glob
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import SimpleITK as sitk

In [47]:
def getLandmarksFromFCSVFile(file):
    df = pd.read_csv(file, header=None, comment="#")
    if df.shape[1] == 6:
        df.columns = ["label", "x", "y", "z", "sel", "vis"]
    elif df.shape[1] == 14:
        df.columns = [
            "id",
            "x",
            "y",
            "z",
            "ow",
            "ox",
            "oy",
            "oz",
            "vis",
            "sel",
            "lock",
            "label",
            "desc",
            "associatedNodeID",
        ]
    else:
        raise "Please check your input FCSV file"
    # the sign flipping in x and y is to convert RAS(used by slicer) to LPS(used in DICOM and itk)
    xcoord = -1 * df["x"].values.reshape(-1, 1)
    ycoord = -1 * df["y"].values.reshape(-1, 1)
    zcoord = df["z"].values.reshape(-1, 1)
    vec = np.concatenate((xcoord, ycoord, zcoord), axis=1).reshape(-1, 3)
    return dict(zip(df["label"], vec))

In [48]:
def getImage(lmksfile, imgfile, name, color):
    fidDict = getLandmarksFromFCSVFile(lmksfile)
    fid = fidDict[name]
    output_size = (30, 30, 1)
    output_spacing = (1, 1, 1) #DOUBT
    offset = np.array([15, 15, 1])
    identity_transform = sitk.Transform(3, sitk.sitkIdentity)
    sitk_image = sitk.ReadImage(imgfile, sitk.sitkFloat32)
    rif = sitk.ResampleImageFilter()
    rif.SetOutputSpacing(output_spacing)
    rif.SetTransform(identity_transform)
    rif.SetOutputOrigin(tuple(np.array(fid) - offset))
    rif.SetOutputDirection([1, 0, 0, 0, 1, 0, 0, 0, 1])
    rif.SetSize(output_size)
    img = rif.Execute(sitk_image)
    img = sitk.RescaleIntensity(
                    img, outputMinimum=0, outputMaximum=255
                )
    img = sitk.GetArrayFromImage(img).T[:, :, 0]
    img = np.dstack((img, img, img))
    img = np.round(img).astype(int)
    img[14, 14, :] = color
    plt.imsave('./images/'+os.path.basename(lmksfile)[:-9]+'_{}.png'.format(name), img)

In [49]:
path = 'YOUR_PATH_HERE'
Fids = ['LE', 'RE']
Colors = [[0, 0, 255], [255, 0, 0]]

In [50]:
if not os.path.isdir('./images/'):
    os.mkdir('./images/')

In [51]:
all_fcsvs = [f for f in glob.glob(path + "**/*_lmks.fcsv", recursive=True)]
with open('report.html','w') as htmlfid:
    htmlfid.write("<!DOCTYPE html>\n")
    htmlfid.write("<html>\n")
    htmlfid.write("<head>\n")
    htmlfid.write("<style>\n")
    htmlfid.write("table, td, th {\n")
    htmlfid.write("border: 1px solid black;\n")
    htmlfid.write("}\n")
    htmlfid.write("</style>\n")
    htmlfid.write("</head>\n")
    htmlfid.write('<body>\n')
    htmlfid.write('<h1>{0}</h1>\n'.format("Fiducial Point Report"))
    htmlfid.write('<p>{0}</p>\n'.format("Author: Arjit Jain (hawkid=arjjain)"))
    htmlfid.write('<p>{0}</p>\n'.format("Fiducial Points Evaluate: {0}".format(", ".join(Fids))))
    htmlfid.write('<table>\n')        
    fids_th_str = ""
    for fid in Fids:
        fids_th_str = "{recur}<th>{fid_th}</th>".format(recur=fids_th_str, fid_th=fid)
    htmlfid.write('<tr><th> subject </th>{th_fids}'.format(th_fids=fids_th_str))
    for i in range(len(all_fcsvs)):
        fid_fn = all_fcsvs[i]
        img_fn = fid_fn[:-9]+'T1w.nii.gz'
        htmlfid.write('  <tr>\n')
        htmlfid.write('     <td>{}</td>\n'.format(os.path.basename(fid_fn)))
        for i in range(len(Fids)):
            getImage(fid_fn, img_fn, Fids[i], Colors[i])
            htmlfid.write('     <td><img src="{}" width="200" height="80"></td>\n'.format('./images/'+os.path.basename(fid_fn)[:-9]+'_{}.png'.format(Fids[i])))
        htmlfid.write('  </tr>\n')
    htmlfid.write('</table>\n')
    htmlfid.write('</body>\n')
    htmlfid.write('</html>\n')